In [1]:
from ultralytics import YOLO

model = YOLO('../yolo11x-pose.pt')

model.export(
    format='engine',
    half=True,
    imgsz=640,
    workspace=8,
    # batch=8,
    # int8=True,
    # data='coco-pose.yaml',
)

WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics 8.3.27 🚀 Python-3.12.7 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24102MiB)
YOLO11x-pose summary (fused): 483 layers, 58,751,308 parameters, 0 gradients, 202.8 GFLOPs

PyTorch: starting from '../yolo11x-pose.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 56, 8400) (113.0 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.42...
ONNX: export success ✅ 2.8s, saved as '../yolo11x-pose.onnx' (224.5 MB)

TensorRT: starting export with TensorRT 10.6.0.post1...
[12/22/2024-13:43:56] [TRT] [I] [MemUsageChange] Init CUDA: CPU +2, GPU +0, now: CPU 1049, GPU 2197 (MiB)
[12/22/2024-13:43:57] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +2276, GPU +438, now: CPU 3481, GPU 2635 (MiB)
[12/22/2024-13:43:57] [TRT] [I] ----------------------------------------------------------------
[12/22/2024-13:43:57] [TRT] [I] Input filename:   ../yolo1

'../yolo11x-pose.engine'

# 3x speedup

In [2]:
import cv2
tensorrt_model = YOLO('../yolo11n-pose.engine')
video_path = '../fall_dataset/videos/video_2.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened:
    print("Error could not open video")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    results = tensorrt_model(frame)
    for result in results:
        frame = result.plot()

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Loading ../yolo11n-pose.engine for TensorRT inference...
[11/03/2024-17:42:35] [TRT] [I] Loaded engine size: 9 MiB
[11/03/2024-17:42:35] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +10, now: CPU 0, GPU 15 (MiB)

0: 640x640 3 persons, 1.4ms
Speed: 1.4ms preprocess, 1.4ms inference, 56.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 2.9ms
Speed: 3.4ms preprocess, 2.9ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 2.7ms
Speed: 2.6ms preprocess, 2.7ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 1.9ms
Speed: 1.1ms preprocess, 1.9ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 1.4ms
Speed: 1.1ms preprocess, 1.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 1.4ms
Speed: 1.2ms preprocess, 1.4ms inference, 0.8ms postprocess per image at shape (